In [1]:
import math
import pickle
import json as js
import numpy as np
import pandas as pd
import scipy.stats as st
from math import sin, cos, sqrt, atan2, radians

In [2]:
!ls '../data/'

df_tdrive_ALL-taxis.csv  exp1-trj.h5	 tdrive.csv		   val.mta
D_p-pts.pickle		 exp1-trj.label  tdrive_formato_t2vec.csv  val.src
D_q-pts.pickle		 exp1-trj.t	 train.mta		   val.trg
Dq-pts.pickle		 mv.csv		 train.src		   vocab.txt
exp1-querydb.h5		 saved_models	 train.trg


In [3]:
df = pd.read_csv('../data/tdrive_formato_t2vec.csv')
df

,Unnamed: 0,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAX_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,0,21,False,False,False,False,False,False,False,"[[116.21163,39.89811],[116.21168,39.89804],[11..."
1,1,22,False,False,False,False,False,False,False,"[[116.21183,39.89739],[116.21186,39.89735],[11..."
2,2,23,False,False,False,False,False,False,False,"[[116.21188,39.89737],[116.2118,39.89743],[116..."
3,3,26,False,False,False,False,False,False,False,"[[116.21183,39.89786],[116.212,39.89805],[116...."
4,4,27,False,False,False,False,False,False,False,"[[116.21122,39.89813],[116.21172,39.89769],[11..."
...,...,...,...,...,...,...,...,...,...,...
532506,532506,3163268,False,False,False,False,False,False,False,"[[116.49875,39.95969],[116.49742,39.96326],[11..."
532507,532507,3163271,False,False,False,False,False,False,False,"[[116.48382,39.97436],[116.47633,39.97806],[11..."
532508,532508,3163274_1,False,False,False,False,False,False,False,"[[116.49705,39.97453],[116.49695,39.97368],[11..."
532509,532509,3163274_2,False,False,False,False,False,False,False,"[[116.88893,40.15263],[116.92498,40.15155],[11..."


In [4]:
# Pegando as trajs (linhas no df) correspondentes as trajs query e dbsearch (querydb)
with open('../data/exp1-trj.label', 'r') as file:
    # Leia as linhas do arquivo e converta para inteiros
    querydb_lines = [int(line.strip()) for line in file]

In [5]:
print(querydb_lines[:100])

[420002, 420003, 420005, 420006, 420007, 420012, 420013, 420016, 420017, 420019, 420021, 420022, 420025, 420031, 420033, 420034, 420036, 420039, 420051, 420052, 420054, 420055, 420057, 420070, 420071, 420075, 420077, 420078, 420080, 420084, 420086, 420089, 420091, 420093, 420094, 420095, 420100, 420101, 420102, 420103, 420104, 420105, 420106, 420107, 420108, 420109, 420110, 420112, 420113, 420114, 420115, 420116, 420117, 420118, 420119, 420120, 420121, 420122, 420123, 420124, 420125, 420126, 420127, 420128, 420129, 420130, 420131, 420133, 420134, 420136, 420137, 420138, 420140, 420141, 420142, 420143, 420144, 420145, 420146, 420149, 420150, 420151, 420152, 420153, 420154, 420155, 420156, 420158, 420159, 420160, 420161, 420162, 420163, 420164, 420165, 420166, 420167, 420168, 420169, 420170]


In [6]:
len(querydb_lines)

50500

In [7]:
# Filtrando apenas a linhas querydb
df = df[df.index.isin(querydb_lines)]
df.reset_index(inplace=True)
del df['index']
df

,Unnamed: 0,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAX_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,420002,2432652,False,False,False,False,False,False,False,"[[116.82171,40.37281],[116.82174,40.3728],[116..."
1,420003,2432653,False,False,False,False,False,False,False,"[[116.82183,40.37366],[116.82164,40.37348],[11..."
2,420005,2432655,False,False,False,False,False,False,False,"[[116.83361,40.4683],[116.83363,40.46828],[116..."
3,420006,2432656,False,False,False,False,False,False,False,"[[116.80888,40.48044],[116.80886,40.48045],[11..."
4,420007,2432659,False,False,False,False,False,False,False,"[[116.80878,40.48052],[116.80878,40.48051],[11..."
...,...,...,...,...,...,...,...,...,...,...
49995,504635,2970415,False,False,False,False,False,False,False,"[[116.30542,40.03542],[116.30727,40.02595],[11..."
49996,504636,2970416,False,False,False,False,False,False,False,"[[116.32396,39.95055],[116.32557,39.95345],[11..."
49997,504637,2970417,False,False,False,False,False,False,False,"[[116.31713,39.89897],[116.31466,39.8959],[116..."
49998,504638,2970418,False,False,False,False,False,False,False,"[[116.31522,39.89797],[116.3153,39.89887],[116..."


In [8]:
# query --> 500 trajs par (Ta)
# dbsearch --> (as 500 trajs ímpar da query (Ta') + 49500 trajs ímpar qualquer dos dados de teste)

In [9]:
querydb_lines[:500]

[420002,
 420003,
 420005,
 420006,
 420007,
 420012,
 420013,
 420016,
 420017,
 420019,
 420021,
 420022,
 420025,
 420031,
 420033,
 420034,
 420036,
 420039,
 420051,
 420052,
 420054,
 420055,
 420057,
 420070,
 420071,
 420075,
 420077,
 420078,
 420080,
 420084,
 420086,
 420089,
 420091,
 420093,
 420094,
 420095,
 420100,
 420101,
 420102,
 420103,
 420104,
 420105,
 420106,
 420107,
 420108,
 420109,
 420110,
 420112,
 420113,
 420114,
 420115,
 420116,
 420117,
 420118,
 420119,
 420120,
 420121,
 420122,
 420123,
 420124,
 420125,
 420126,
 420127,
 420128,
 420129,
 420130,
 420131,
 420133,
 420134,
 420136,
 420137,
 420138,
 420140,
 420141,
 420142,
 420143,
 420144,
 420145,
 420146,
 420149,
 420150,
 420151,
 420152,
 420153,
 420154,
 420155,
 420156,
 420158,
 420159,
 420160,
 420161,
 420162,
 420163,
 420164,
 420165,
 420166,
 420167,
 420168,
 420169,
 420170,
 420171,
 420172,
 420173,
 420174,
 420175,
 420176,
 420178,
 420180,
 420181,
 420182,
 420183,
 

In [10]:
querydb_lines[500:1000] # ínicio do dbsearch começa com as Ta'!

[420002,
 420003,
 420005,
 420006,
 420007,
 420012,
 420013,
 420016,
 420017,
 420019,
 420021,
 420022,
 420025,
 420031,
 420033,
 420034,
 420036,
 420039,
 420051,
 420052,
 420054,
 420055,
 420057,
 420070,
 420071,
 420075,
 420077,
 420078,
 420080,
 420084,
 420086,
 420089,
 420091,
 420093,
 420094,
 420095,
 420100,
 420101,
 420102,
 420103,
 420104,
 420105,
 420106,
 420107,
 420108,
 420109,
 420110,
 420112,
 420113,
 420114,
 420115,
 420116,
 420117,
 420118,
 420119,
 420120,
 420121,
 420122,
 420123,
 420124,
 420125,
 420126,
 420127,
 420128,
 420129,
 420130,
 420131,
 420133,
 420134,
 420136,
 420137,
 420138,
 420140,
 420141,
 420142,
 420143,
 420144,
 420145,
 420146,
 420149,
 420150,
 420151,
 420152,
 420153,
 420154,
 420155,
 420156,
 420158,
 420159,
 420160,
 420161,
 420162,
 420163,
 420164,
 420165,
 420166,
 420167,
 420168,
 420169,
 420170,
 420171,
 420172,
 420173,
 420174,
 420175,
 420176,
 420178,
 420180,
 420181,
 420182,
 420183,
 

In [11]:
def split_Tas(trajetorias):
    """
    Divide as trajetórias passadas em Ta e Ta'; Ta --> pontos(lon, lat) ímpares, Ta' --> pontos(lon, lat) pares)
    Input: Lista de trajetorias, trajetórias representadas pela sequencia de pontos (GPS)
    Output: Lista de trajetórias Ta e lista de trajetórias Ta'
    """
    Dq = []
    D_q = []
    for j in range(len(trajetorias)):
        Ta = []
        Ta_ = []
        if len(trajetorias[j]) == 1: # Se a traj tiver comprimento 1, a traj par será ela mesma!
            Ta.append(trajetorias[j][0])
            Ta_.append(trajetorias[j][0])
        else:
            for i in range(0,len(trajetorias[j]), 2): # pega os impares
                Ta.append(trajetorias[j][i])
            for i in range(1,len(trajetorias[j]), 2): # pega os pares
                Ta_.append(trajetorias[j][i])
        Dq.append(np.array(Ta))
        D_q.append(np.array(Ta_))

    return Dq, D_q

In [12]:
trajs_teste = df.POLYLINE.to_list()

In [13]:
trajs_teste[:5]

['[[116.82171,40.37281],[116.82174,40.3728],[116.82175,40.37281],[116.82171,40.37251],[116.82152,40.37284],[116.82152,40.3722],[116.82182,40.37286],[116.82178,40.37288],[116.82177,40.37284],[116.82183,40.37386],[116.82184,40.37366]]',
 '[[116.82183,40.37366],[116.82164,40.37348],[116.82168,40.37307],[116.8217,40.37305],[116.82175,40.37271],[116.82176,40.37273],[116.82176,40.37276],[116.82178,40.37278]]',
 '[[116.83361,40.4683],[116.83363,40.46828],[116.83363,40.46827],[116.83593,40.47344],[116.80887,40.4804],[116.80887,40.48053],[116.80887,40.48052],[116.80887,40.48051],[116.80889,40.4805],[116.80891,40.48048],[116.80891,40.48047],[116.80893,40.48046],[116.80894,40.48045],[116.80894,40.48044],[116.80893,40.48044],[116.80892,40.48044],[116.80891,40.48044],[116.8089,40.48044],[116.8089,40.48045],[116.80892,40.48044],[116.80891,40.48042],[116.80884,40.48049],[116.80884,40.48053],[116.80883,40.48054],[116.80884,40.48054],[116.80884,40.48053],[116.80884,40.48052],[116.80883,40.48052],[116.8

In [14]:
type(trajs_teste[1])

str

In [15]:
# Convertendo de lista de strings p/ lista de listas
tmp = []
for string in trajs_teste:
    json_list = js.loads(string)
    tmp.append(json_list)

trajs_teste = tmp

In [16]:
trajs_teste[:5]

[[[116.82171, 40.37281],
  [116.82174, 40.3728],
  [116.82175, 40.37281],
  [116.82171, 40.37251],
  [116.82152, 40.37284],
  [116.82152, 40.3722],
  [116.82182, 40.37286],
  [116.82178, 40.37288],
  [116.82177, 40.37284],
  [116.82183, 40.37386],
  [116.82184, 40.37366]],
 [[116.82183, 40.37366],
  [116.82164, 40.37348],
  [116.82168, 40.37307],
  [116.8217, 40.37305],
  [116.82175, 40.37271],
  [116.82176, 40.37273],
  [116.82176, 40.37276],
  [116.82178, 40.37278]],
 [[116.83361, 40.4683],
  [116.83363, 40.46828],
  [116.83363, 40.46827],
  [116.83593, 40.47344],
  [116.80887, 40.4804],
  [116.80887, 40.48053],
  [116.80887, 40.48052],
  [116.80887, 40.48051],
  [116.80889, 40.4805],
  [116.80891, 40.48048],
  [116.80891, 40.48047],
  [116.80893, 40.48046],
  [116.80894, 40.48045],
  [116.80894, 40.48044],
  [116.80893, 40.48044],
  [116.80892, 40.48044],
  [116.80891, 40.48044],
  [116.8089, 40.48044],
  [116.8089, 40.48045],
  [116.80892, 40.48044],
  [116.80891, 40.48042],
  [116

In [17]:
type(trajs_teste[1])

list

In [18]:
len(trajs_teste)

50000

In [19]:
# Fazendo os splits (trajs par e ímpar)
query = trajs_teste[:500] # 500 trajetórias query
Dq, D_q = split_Tas(query)
db = trajs_teste[500:50000] # 49_500 trajetórias do dbsearch
Dp, D_p = split_Tas(db)

In [20]:
path = '../data/'

In [21]:
def haversine(p1, p2):
    # Raio médio da Terra em metros
    earth_radius = 6371000.0

    # Conversão de graus para radianos
    lat1 = math.radians(p1[0])
    lon1 = math.radians(p1[1])
    lat2 = math.radians(p2[0])
    lon2 = math.radians(p2[1])

    # Diferença de latitudes e longitudes
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula de haversine
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Distância em metros
    distance = earth_radius * c
    return distance

In [22]:
p1 = np.array([-37.989553, -4.949479], dtype=np.float32)
p2 = np.array([-37.992316, -4.9481 ], dtype=np.float32)
print(haversine(p1, p2))
print(haversine(p1, p2))

330.4194160706921
330.4194160706921


In [23]:
# Quanto mais próximo de zero, mais parecidas são as trajetórias!
def dist_edr_trajs(t1, t2):
    """ Edit Distance on Real Sequence"""
    """ The edit distance between two strings refers to the minimum number of character insertions, deletions, and substitutions required to change one string to the other."""

    thr = 100 # threshold in meters

    if len(t1) > len(t2):
        difference = len(t1) - len(t2)
        for i in range(len(t2)):
          if haversine(t1[i], t2[i]) > thr:
            difference += 1

    elif len(t2) > len(t1):
        difference = len(t2) - len(t1)
        for i in range(len(t1)):
          if haversine(t1[i], t2[i]) > thr:
            difference += 1

    else:
        difference = 0
        for i in range(len(t1)):
          if haversine(t1[i], t2[i]) > thr:
            difference += 1


    return difference

In [24]:
t1 =  np.array([[-37.97151, -4.93422],
                [-37.971341, -4.934051],
                [-37.971111, -4.933850],
                [-37.970728, -4.933544],
                [-37.970429, -4.933256],
                [-37.970207, -4.933063]], dtype=np.float32)

t2 =  np.array([[-37.97135, -4.93438],
                [-37.971143, -4.934198],
                [-37.970769, -4.933945],
                [-37.969606, -4.934348]], dtype=np.float32)

In [25]:
print(haversine(t1[0], t2[0]))
print(haversine(t1[1], t2[1]))
print(haversine(t1[2], t2[2]))
print(haversine(t1[3], t2[3]))

22.685346133327865
25.539213148204052
39.08032594212811
143.61174704597673


In [26]:
dist_edr_trajs(t1, t2)

3

In [27]:
Dq[0]

array([[116.82171,  40.37281],
       [116.82175,  40.37281],
       [116.82152,  40.37284],
       [116.82182,  40.37286],
       [116.82177,  40.37284],
       [116.82184,  40.37366]])

In [28]:
D_q[0]

array([[116.82174,  40.3728 ],
       [116.82171,  40.37251],
       [116.82152,  40.3722 ],
       [116.82178,  40.37288],
       [116.82183,  40.37386]])

In [29]:
D_p[0]

array([[116.41468,  39.87831],
       [116.41383,  39.88515],
       [116.42151,  39.89236],
       [116.43186,  39.89213],
       [116.43391,  39.88984]])

In [30]:
haversine(Dq[0][0], D_q[0][0])

3.3733681642732756

In [31]:
# Salvando as trajs points em arquivos para semrem usadas nas demais técnicas...

# Salvando Dq:
with open(path+'Dq-pts.pickle', 'wb') as file:
    pickle.dump(Dq, file)

# Salvando D_q:
with open(path+'D_q-pts.pickle', 'wb') as file:
    pickle.dump(D_q, file)

# Salvando D_p:
with open(path+'D_p-pts.pickle', 'wb') as file:
    pickle.dump(D_p, file)

In [32]:
def rank(t_query, i, db_search):
    dists = list(map(lambda t: dist_edr_trajs(t_query, t), db_search)) # pega todas as distâncias de Dq[i] as trajs de D_qUD_p
    dists = np.array(dists)
    order = dists.argsort() # pega a ordem
    ranks = order.argsort() # pega o rank

    return ranks[i] + 1 # retorna o ranking de ta' no dbsearch | soma +1 pq o argsort rankea a partir de 0 (zero)

In [33]:
def acc(ranks):
    count = 0
    for i in range(len(ranks)):
        if ranks[i] == 1:
            count += 1

    return round(count/len(ranks), 3)

In [34]:
def mr(ranks):
    return (sum(ranks)/len(ranks))

In [35]:
def mrr(ranks):
    count = 0
    for i in range(len(ranks)):
        count += 1/ranks[i]

    return round(count/len(ranks), 3)

In [36]:
# Intervalo de Confiança do Ranks
def cip_r(ranks):
    data = []
    for i in range(len(ranks)):
        data.append(ranks[i]) # Add os Ranks
    
    #create 95% confidence interval for population mean weight
    ic = st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))
    
    return tuple(round(valor, 3) for valor in ic) # arredonda pra 3 casas decimais

In [37]:
# Intervalo de Confiança dos Reciprocal Ranks
def cip_rr(ranks):
    data = []
    for i in range(len(ranks)):
        data.append(1/ranks[i]) # Add os RRs...
    
    #create 95% confidence interval for population mean weight
    ic = st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))
    
    return tuple(round(valor, 3) for valor in ic) # arredonda pra 3 casas decimais

In [46]:
%%time
dbsizes = [10000, 20000, 30000, 40000, 50000]
entire_db = D_q + D_p
for dbsize in dbsizes:
    ranks = []
    dbsearch = entire_db[:dbsize]
    for i in range(len(query)):
        ranks.append(rank(Dq[i], i, dbsearch)) # rank de ta' no dbsearch!
    print('Mean rank: {}{}, Acc: {}, MRR: {}{} with dbsearch size: {}'.format(mr(ranks), cip_r(ranks), acc(ranks), mrr(ranks), cip_rr(ranks), dbsize))

Mean rank: 587.558(482.466, 692.65), Acc: 0.592, MRR: 0.655(0.616, 0.693) with dbsearch size: 10000
Mean rank: 1193.976(983.721, 1404.231), Acc: 0.574, MRR: 0.634(0.595, 0.672) with dbsearch size: 20000
Mean rank: 2089.506(1730.156, 2448.856), Acc: 0.572, MRR: 0.63(0.59, 0.669) with dbsearch size: 30000
Mean rank: 2693.076(2213.075, 3173.077), Acc: 0.564, MRR: 0.624(0.585, 0.663) with dbsearch size: 40000
Mean rank: 3472.314(2851.48, 4093.148), Acc: 0.548, MRR: 0.608(0.569, 0.648) with dbsearch size: 50000
CPU times: user 27min 6s, sys: 450 ms, total: 27min 6s
Wall time: 32min 46s
